In [1]:
import warnings

import geopandas
import libpysal
import momepy
import osmnx
import pandas

from clustergram import Clustergram

import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

ModuleNotFoundError: No module named 'geopandas'

In [2]:
place = 'Kyiv, Ukraine'
local_crs = 5514

In [4]:
geopandas.tools.geocode(place).explore()

In [5]:
buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})
buildings.head()

/var/folders/2q/v85mb9ld0sd53grpcrrhdv980000gn/T/ipykernel_36757/1465348557.py:1: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})


name                  name:uk operator  \
element_type osmid                                                            
node         385664630            Pacific                  Pacific      NaN   
             660430209   Іллінська церква  Іллінська церква УПЦ МП      NaN   
             796648457                КПП                      КПП      NaN   
             835130259                NaN                      NaN      NaN   
             1119810191               NaN                      NaN      NaN   

                        railway  ref wheelchair                   geometry  \
element_type osmid                                                           
node         385664630      NaN  NaN        NaN  POINT (30.53564 50.44551)   
             660430209      NaN  NaN        NaN  POINT (30.52420 50.46661)   
             796648457      NaN  NaN        NaN  POINT (30.54978 50.39972)   
             835130259      NaN  NaN        NaN  POINT (30.51546 50.42443)   
             1119810191     NaN  NaN        NaN  POINT (30.51539 50.42454)   

                        entrance man_made           amenity  ...  \
element_type osmid                                           ...   
node         385664630       NaN      NaN        restaurant  ...   
             660430209       NaN      NaN  place_of_worship  ...   
             796648457       NaN      NaN               NaN  ...   
             835130259       NaN      NaN               NaN  ...   
             1119810191      NaN      NaN               NaN  ...   

                        toilets:handwashing addr:block playground size levels  \
element_type osmid                                                              
node         385664630                  NaN        NaN        NaN  NaN    NaN   
             660430209                  NaN        NaN        NaN  NaN    NaN   
             796648457                  NaN        NaN        NaN  NaN    NaN   
             835130259                  NaN        NaN        NaN  NaN    NaN   
             1119810191                 NaN        NaN        NaN  NaN    NaN   

                        ways abandoned:shop website_1 ref:edbo  \
element_type osmid                                               
node         385664630   NaN            NaN       NaN      NaN   
             660430209   NaN            NaN       NaN      NaN   
             796648457   NaN            NaN       NaN      NaN   
             835130259   NaN            NaN       NaN      NaN   
             1119810191  NaN            NaN       NaN      NaN   

                        payment:discover_card  
element_type osmid                             
node         385664630                    NaN  
             660430209                    NaN  
             796648457                    NaN  
             835130259                    NaN  
             1119810191                   NaN  

[5 rows x 578 columns]

In [6]:
buildings.geom_type.value_counts()

Polygon         89227
Point             499
MultiPolygon        7
LineString          3
Name: count, dtype: int64

In [7]:
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)


In [9]:
buildings = buildings[["geometry"]].to_crs(local_crs)

In [10]:
buildings["uID"] = range(len(buildings))
buildings.head()

,geometry,uID
0,"POLYGON ((400715.451 -1047066.528, 400798.992 ...",0
1,"POLYGON ((411512.314 -1039314.466, 411523.660 ...",1
2,"POLYGON ((411551.985 -1039399.946, 411563.241 ...",2
3,"POLYGON ((411535.943 -1039300.928, 411574.307 ...",3
4,"POLYGON ((411589.050 -1039278.554, 411600.700 ...",4


In [11]:
osm_graph = osmnx.graph_from_place(place, network_type='drive')
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)

In [12]:
streets.head()

osmid  oneway lanes                   name  \
u          v          key                                                   
10980673   2522195122 0      4357040    True     2  Короленківська вулиця   
           10980674   0    182241803    True     4       Жилянська вулиця   
2522195122 26546563   0      4357040    True     2  Короленківська вулиця   
           2522195124 0    745961034    True   NaN                    NaN   
10980674   26201486   0     27030896    True     2     Тарасівська вулиця   

                                  highway maxspeed reversed   length  \
u          v          key                                              
10980673   2522195122 0         secondary       50    False   77.989   
           10980674   0           primary       50    False   63.325   
2522195122 26546563   0         secondary       50    False   14.317   
           2522195124 0    secondary_link      NaN    False   10.315   
10980674   26201486   0          tertiary      NaN    False  218.741   

                                                                    geometry  \
u          v          key                                                      
10980673   2522195122 0    LINESTRING (402991.403 -1040121.999, 402997.91...   
           10980674   0    LINESTRING (402991.403 -1040121.999, 403000.04...   
2522195122 26546563   0    LINESTRING (403032.862 -1040188.176, 403039.87...   
           2522195124 0    LINESTRING (403032.862 -1040188.176, 403036.00...   
10980674   26201486   0    LINESTRING (403052.464 -1040137.406, 403054.95...   

                          bridge tunnel  ref junction access width est_width  
u          v          key                                                     
10980673   2522195122 0      NaN    NaN  NaN      NaN    NaN   NaN       NaN  
           10980674   0      NaN    NaN  NaN      NaN    NaN   NaN       NaN  
2522195122 26546563   0      NaN    NaN  NaN      NaN    NaN   NaN       NaN  
           2522195124 0      NaN    NaN  NaN      NaN    NaN   NaN       NaN  
10980674   26201486   0      NaN    NaN  NaN      NaN    NaN   NaN       NaN

In [13]:
streets = momepy.remove_false_nodes(streets)
streets = streets[["geometry"]]
streets["nID"] = range(len(streets))

/Users/petrokvartsianyi/opt/miniconda3/envs/scientificProject1/lib/python3.10/site-packages/geopandas/array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as S-JTSK / Krovak East North (the single non-null crs provided).
  warnings.warn(


In [14]:
streets.head()


,geometry,nID
0,"LINESTRING (402991.403 -1040121.999, 402997.91...",0
1,"LINESTRING (402991.403 -1040121.999, 403000.04...",1
2,"LINESTRING (403032.862 -1040188.176, 403039.87...",2
3,"LINESTRING (403032.862 -1040188.176, 403036.00...",3
4,"LINESTRING (403052.464 -1040137.406, 403054.95...",4


# Generated data

## Tessellation

In [ ]:
limit = momepy.buffered_limit(buildings, 100)

tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=False, segment=1)
tessellation = tessellation.tessellation

## Link streets

In [ ]:
buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
buildings.head()

In [ ]:
buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

# Measure

## Dimensions

In [ ]:
buildings["area"] = buildings.area
tessellation["area"] = tessellation.area
streets["length"] = streets.length

## Shape

In [ ]:
buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series
buildings['elongation'] = momepy.Elongation(buildings).series
tessellation['convexity'] = momepy.Convexity(tessellation).series
streets["linearity"] = momepy.Linearity(streets).series

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("eri", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("elongation", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("linearity", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

## Spatial distribution

In [ ]:
buildings["shared_walls"] = momepy.SharedWallsRatio(buildings).series
buildings.plot("shared_walls", figsize=(12, 12), scheme="natural_breaks", legend=True).set_axis_off()

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)

In [ ]:
tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    buildings["neighbor_distance"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("neighbor_distance", ax=ax[0], scheme="natural_breaks", legend=True)
tessellation.plot("covered_area", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
queen_3 = momepy.sw_high(k=3, weights=queen_1)
buildings_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings, silence_warnings=True)

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', queen_3, verbose=False).series
buildings['adjacency'] = momepy.BuildingAdjacency(buildings, queen_3, 'uID', buildings_q1, verbose=False).series

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("interbuilding_distance", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("adjacency", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))

streets.plot("width", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("width_deviation", ax=ax[1], scheme="natural_breaks", legend=True)
streets.plot("openness", ax=ax[2], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

## Intensity

In [ ]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series
tessellation.plot("car", figsize=(12, 12), vmin=0, vmax=1, legend=True).set_axis_off()

## Connectivity

In [ ]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")
nodes, streets = momepy.nx_to_gdf(graph)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))

nodes.plot("degree", ax=ax[0], scheme="natural_breaks", legend=True, markersize=1)
nodes.plot("closeness", ax=ax[1], scheme="natural_breaks", legend=True, markersize=1, legend_kwds={"fmt": "{:.6f}"})
nodes.plot("meshedness", ax=ax[2], scheme="natural_breaks", legend=True, markersize=1)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

In [ ]:
buildings["nodeID"] = momepy.get_node_id(buildings, nodes, streets, "nodeID", "nID")

In [ ]:
# Link all data together (to tessellation cells or buildings).

merged = tessellation.merge(buildings.drop(columns=['nID', 'geometry']), on='uID')
merged = merged.merge(streets.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')
merged.columns

# Understanding the context

In [ ]:
# Measure first, second and third quartile of distribution of values within an area around each building.
percentiles = []
for column in merged.columns.drop(["uID", "nodeID", "nID", 'mm_len', 'node_start', 'node_end', "geometry"]):
    perc = momepy.Percentiles(merged, column, queen_3, "uID", verbose=False).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

In [ ]:
percentiles_joined = pandas.concat(percentiles, axis=1)
percentiles_joined.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
merged.plot(percentiles_joined['convexity_50'].values, ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()